<a href="https://colab.research.google.com/github/jcbolo72012/Deep-Learning-For-Music/blob/master/MusicAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Data Initialization 
import torch
import _pickle as cPickle
from torch.utils.data import Dataset
import copy
class MusicDataset(Dataset):
    """Music dataset."""

    def __init__(self, pickle_file, split='train'):
      super().__init__()
      file = open(pickle_file, 'rb')
      self.dataset = cPickle.load(file)[split]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        #song is a list of timesteps, which is a list containing notes
        song = []
        
        #Create a blank 87 note tensor (index 0 is note #21 and index 86 is note #108)
        tensor = []
        for i in range(87):
          tensor += [0]
        # print(self.dataset.keys())
        for timestep in self.dataset[idx]:
          
          temp_TimeStep = copy.deepcopy(tensor)
          for note in timestep:
            temp_TimeStep[note-21] = 1
          song.append(temp_TimeStep)

        return torch.tensor(song)


In [ ]:
#Training and Regression
from torch.utils.data import DataLoader
import torch.optim as optim

dataset = MusicDataset('MuseData.pickle')
dataloader = DataLoader(dataset, batch_size=1)

lstm = torch.nn.LSTM(87, 87)
lossFct = torch.nn.MSELoss()
optimizer = torch.optim.SGD(lstm.parameters(), lr=0.001, momentum=0.9)
history = []
for epoch in range(100):
  for i, batch in enumerate(dataloader):
    # print(batch.shape)
    batch = batch.float()

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    output, _ = lstm(batch)
    # print(output.shape)
    # # print(output[0,0])
    # print(batch.shape)
    # output = output.permute(0, 2, 1)
    # batch = batch.permute(0, 2, 1)
    loss = lossFct(output[:, :-1], batch[:, 1:])
    loss.backward()
    optimizer.step()

    history.append(float(loss))
    # print statistics
    # running_loss += loss.item()
    if i % 2000 == 1999:    # print every 2000 mini-batches
        print(float(loss))
    #           (epoch + 1, i + 1, running_loss / 2000))
    #     running_loss = 0.0

torch.save(lstm, 'lstm.pt')
print('Finished Training')
print(history)

Finished Training
[0.04744202643632889, 0.04301981255412102, 0.03416167199611664, 0.033399444073438644, 0.030805855989456177, 0.025241974741220474, 0.019092679023742676, 0.017648890614509583, 0.046563051640987396, 0.03506388142704964, 0.032872892916202545, 0.030356282368302345, 0.04603354260325432, 0.048368681222200394, 0.04746358096599579, 0.035092081874608994, 0.033419668674468994, 0.031866654753685, 0.03235200047492981, 0.048492129892110825, 0.03362566605210304, 0.029265770688652992, 0.020460058003664017, 0.016738492995500565, 0.029679711908102036, 0.012943902052938938, 0.032566070556640625, 0.04505501314997673, 0.0442962720990181, 0.04802151396870613, 0.04775797575712204, 0.03847143054008484, 0.02204301953315735, 0.026704734191298485, 0.04858516901731491, 0.033918559551239014, 0.05875442922115326, 0.04369402676820755, 0.062008246779441833, 0.05672852322459221, 0.054100487381219864, 0.03654904291033745, 0.04881180822849274, 0.04047315567731857, 0.023876840248703957, 0.05814830586314

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type LSTM. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [ ]:
type(dataset)
dataset.keys()
dataset['train']
#dataset[set][song][timestep][note(s)]

In [ ]:
#Graphing Decreasing Loss
import matplotlib.pyplot as plt
import numpy as np

y = np.asarray(history)
x = np.arange(len(history))

colors = (0,0,0)
area = np.pi*3

# Plot
plt.scatter(x, y, s=area, c=colors, alpha=0.2)
plt.title('Scatter plot pythonspot.com')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.plot(np.unique(x), np.poly1d(np.polyfit(x, y, 1))(np.unique(x)))
plt.grid()
plt.show()